# Práctica 3b: Caracterización y Clasificación de Texturas
## Gabriel Daniel Aguilar Luna, Zuriel Uzai Rodrígez Agiss
### _Facultad de Ingenierría, Universidad Nacional Autónoma de México_
### _Ciudad de México, México_
#### gabriel.aguilar@ingenieria.unam.edu,  zurieluzai2015@gmail.com

***

### __1. Objetivos__

El alumno:
- Desarrollará métodos de caracterización de texturas
- Aprenderáa utilizar clasificadores como K-NN,K-Means omáquinas de soporte vectoria

***

### __3. Desarrollo__

##### Imports y funciones

In [1]:
import matplotlib.pyplot as plt
import skimage as ski
import numpy as np
from shapely.geometry import Polygon, MultiPolygon
from descartes import PolygonPatch
from math import log
from sklearn.neighbors import KNeighborsClassifier
from skimage.segmentation import slic

# Esta funcion 'desdobla' cadenas para crear un rango y un numero
# eg: '100'=mayores que 100, '0-50'= entre 0 y 50, '10-20,500'= entre 10 y 20 o mayores de 500
def listRanger(rango: str):
    # Separa la cadena por comas
    rango = rango.split(',')
    # Definicion de las variables de retorno
    listarango = []
    mayorq = -1
    # Recorre las sentencias separadas anteriormente
    for x in rango:
        # Si la sentencia tiene un '-' es un rango
        if '-' in x:
            # Los rangos se añaden a la lista listarango
            listax = x.split('-')
            listarango += list(range(int(listax[0]),int(listax[1])))
        # Si no, es una cota inferior
        else:
            # Solo puede existir una de estas cotas en la sentencia
            mayorq = int(x)

    return [listarango, mayorq]

# Esta función imprime la imagen junto con las curvas de la posición nColl de contornos
# Que cumplan con range y retorna un arreglo con dichas curvas.
# Se puede especificar los puntos en el perimetro de las lineas con range. eg: range='400-560'
# Si no se especifica se utiliza '150'
# Se puede especificar un nombre para guardar la imagen resultante con save. eg: save='imagen2.png'
# Si no se especifica se utiliza 'ImCrTMP.png'
# eg: masker.printImCr(fruta, fruta_contornos, 1, range='200', save='comida_contornos.png')
def printImCr(imagen, contornos, nColl, **kwargs):
    # Desdobla range
    rangolstd = listRanger(kwargs["range"] if ("range" in kwargs) else '150')
    # Define variable de retorno
    curvas_array = []
    # Con este for se muestran todas las lineas cuya primera dimension entre en el rango
    for i in contornos.collections[nColl].get_paths():
        # Si cumple con las caracteristicas indicadas en range se añade al plot y a la var de retorno
        if len(i.vertices) in rangolstd[0] or ((len(i.vertices) > rangolstd[1]) if (rangolstd[1] != -1) else (len(i.vertices) > len(i.vertices)+1)):
            plt.plot(i.vertices[:,0], i.vertices[:,1], '--b')
            curvas_array.append(i.vertices)
    # Muestra la imagen
    plt.imshow(imagen)
    plt.axis('off')
    # Salva la imagen
    plt.savefig('resultados/'+kwargs["save"] if ("save" in kwargs) else 'resultados/ImCrTMP.png', bbox_inches='tight', transparent=False, pad_inches = 0)
    # Regresa el arreglo
    return curvas_array

# Esta función imprime la imagen de fondo junto con los poligonos
# Definidos por las curvas en curvas_arr
# Retorna un obj Multipolygon
def printImPoly(curvas_arr, fondo):
    # Arreglo aux
    poly_array = []
    # Recorre arreglo de curvas
    for crvua in curvas_arr:
        x = crvua[:,0]
        y = crvua[:,1]
        poly_array.append(Polygon([(i[0], i[1]) for i in zip(x,y)]))
    polygons = MultiPolygon(poly_array)
    #len(polygons.geoms)
    #polygons
    fig = plt.figure() 
    ax = fig.gca()

    # Plotea la imagen de fondo
    plt.imshow(fondo)
    # Plotea los poligonos
    for poly in polygons:
        ax.add_patch(PolygonPatch(poly))
    #ax.axis('scaled')
    plt.show()
    # Retorna los poligonos
    return polygons

# Calcula el valor SIGMA de una matriz
def getSigma(matriz_pix):
    MU = np.mean(matriz_pix, axis=0)
    SIGMA = np.zeros((len(MU),len(MU)))

    for pixel in matriz_pix:
        P_MU = np.array([pixel-MU])
        SIGMA += np.dot(np.transpose(P_MU),P_MU)
    
    return SIGMA / len(matriz_pix)

# Calcula la probabilidad de una clase
def probaClase(lenClase, x, y, n):
    return lenClase/(x*y*n)

# Calcula la parte derecha de la clasificación
def ladoDerecho(SIGMA_det, probaClase):
    return (-log(SIGMA_det)/2) + log(probaClase)

# Calcula la probabilidad de que un pixel pertenezca a una clase
def probaPixClase(pixel, MU, SIGMA_inv):
    P_MU = np.array([pixel-MU])
    #print((np.dot(np.dot(P_MU,SIGMA_inv), np.transpose(P_MU))[0,0])/-2)
    return (np.dot(np.dot(P_MU,SIGMA_inv), np.transpose(P_MU))[0,0])/-2

# Clasifica los pixeles de una imagen en las clases dadas.
def clasificador(imagen, MUs, SIGMAs_inv, ldDers, colores, **kwargs):
    show = (kwargs["show"] if ("show" in kwargs) else False)
    if (isinstance(imagen, str)):
        prueba_img = io.imread(imagen)
        prueba = np.array(prueba_img)
    elif(isinstance(imagen, np.ndarray)):
        prueba = imagen
    if show:    
        plt.imshow(prueba)
        plt.show()
    
    dimensiones = prueba.shape
    resultado = np.zeros((dimensiones[0],dimensiones[1],3), dtype=int)
    for i in range(len(prueba)):
        for j in range(len(prueba[i])):
            arr_aux = []
            for n in range(len(MUs)):
                arr_aux.append(probaPixClase(prueba[i][j], MUs[n], SIGMAs_inv[n])+ldDers[n])
            resultado[i][j]= colores[arr_aux.index(max(arr_aux))]
    return resultado

# Lista de texturas a utilizar.
def listaTexturas(text_names):
    texturas_array = []
    for text_name in text_names:
        objeto = io.imread('texturas/D'+text_name+'.bmp')
        texturas_array.append(np.array(objeto)[:,:,0])
        plt.imshow(objeto)
        plt.axis('off')
        plt.show()
    return texturas_array

### Funciones de obtención de matrices GLCM

In [2]:
# GLCM horizaontal
def glcmMaker_h(ventana, norm = False): 
    matriz_auxiliar = np.zeros((np.amax(ventana)+1,)*2, dtype=int)
    Y, X = ventana.shape
    normalizador = 0
    for renglon in ventana:
        for j in range(X-1):
            matriz_auxiliar[renglon[j]][renglon[j+1]] += 1
            normalizador += 2
    glcm = matriz_auxiliar + np.transpose(matriz_auxiliar)
    return (glcm/normalizador) if norm else glcm

# GLCM vertical
def glcmMaker_v(ventana, norm = False): 
    matriz_auxiliar = np.zeros((np.amax(ventana)+1,)*2, dtype=int)
    Y, X = ventana.shape
    normalizador = 0
    for i in range(Y-1):
        for j in range(X):
            matriz_auxiliar[ventana[i][j]][ventana[i+1][j]] += 1
            normalizador += 2
    glcm = matriz_auxiliar + np.transpose(matriz_auxiliar)
    return (glcm/normalizador) if norm else glcm

# GLCM diagonal (esq.inf.izq -> esq.sup.der.)
def glcmMaker_45(ventana, norm = False): 
    matriz_auxiliar = np.zeros((np.amax(ventana)+1,)*2, dtype=int)
    Y, X = ventana.shape
    normalizador = 0
    for i in range(1,Y):
        for j in range(X-1):
            matriz_auxiliar[ventana[i][j]][ventana[i-1][j+1]] += 1
            normalizador += 2
    glcm = matriz_auxiliar + np.transpose(matriz_auxiliar)
    return (glcm/normalizador) if norm else glcm

# GLCM diagonal (esq.sup.izq. -> esq.inf.der.)
def glcmMaker_135(ventana, norm = False): 
    matriz_auxiliar = np.zeros((np.amax(ventana)+1,)*2, dtype=int)
    Y, X = ventana.shape
    normalizador = 0
    for i in range(1,Y):
        for j in range(1,X):
            matriz_auxiliar[ventana[i][j]][ventana[i-1][j-1]] += 1
            normalizador += 2
    glcm = matriz_auxiliar + np.transpose(matriz_auxiliar)
    return (glcm/normalizador) if norm else glcm

### Funciones de obtención de características

In [3]:
def entropy(glcm_n):
    entropia = 0
    for renglon in glcm_n:
        for value in renglon:
            entropia += (value*log(value)) if (value>0) else 0
    return -entropia

def homogeneity(glcm_n):
    homogeneidad = 0
    Y, X = glcm_n.shape
    for i in range(Y):
        for j in range(X):
            homogeneidad += glcm_n[i][j]/(1+abs(i-j))
    return homogeneidad

#Angular Second Moment, Energy
def smoothness(glcm_n):
    ASM = 0
    Y, X = glcm_n.shape
    for i in range(Y):
        for j in range(X):
            ASM += glcm_n[i][j]**2
    return ASM

def contrast(glcm_n,k=2,n=1):
    contraste = 0
    Y, X = glcm_n.shape
    for i in range(Y):
        for j in range(X):
            contraste += ((i-j)**k)*(glcm_n[i][j]**n)
    return contraste

In [4]:
# Recibe una imagen y un tamaño de ventana,
# Calcula la entropia para cada ventana de la imagen
# Regresa un arreglo en forma de imagen concatenando los resultados de cada ventana.
def imagenWEntropy(imagen,window_length,window_height,**kwargs):
    Y, X = imagen.shape
    array_aux = []
    length_overlap = (kwargs["length_overlap"] if ("length_overlap" in kwargs) else 0)
    height_overlap = (kwargs["height_overlap"] if ("height_overlap" in kwargs) else 0)
    mostrar = (kwargs["show"] if ("show" in kwargs) else False)
    for texel_renglon in range(0,Y,window_height-height_overlap):
        renglon_aux = []
        for texel_startP in range(0,X,window_length-length_overlap):
            ventana = imagen[texel_renglon:texel_renglon+window_height,texel_startP:texel_startP+window_length]
            glcm_N = glcmMaker_h(ventana,True)
            renglon_aux.append(entropy(glcm_N))
        array_aux.append(renglon_aux)
    imagen_res = np.array(array_aux)
    if mostrar:
        plt.imshow(imagen_res)
        plt.axis('off')
        plt.show()
    return imagen_res

# Recibe una imagen y un tamaño de ventana,
# Calcula la homogeneidad para cada ventana de la imagen
# Regresa un arreglo en forma de imagen concatenando los resultados de cada ventana.
def imagenWHomogeneity(imagen,window_length,window_height,**kwargs):
    Y, X = imagen.shape
    array_aux = []
    length_overlap = (kwargs["length_overlap"] if ("length_overlap" in kwargs) else 0)
    height_overlap = (kwargs["height_overlap"] if ("height_overlap" in kwargs) else 0)
    mostrar = (kwargs["show"] if ("show" in kwargs) else False)
    for texel_renglon in range(0,Y,window_height-height_overlap):
        renglon_aux = []
        for texel_startP in range(0,X,window_length-length_overlap):
            ventana = imagen[texel_renglon:texel_renglon+window_height,texel_startP:texel_startP+window_length]
            glcm_N = glcmMaker_h(ventana,True)
            renglon_aux.append(homogeneity(glcm_N))
        array_aux.append(renglon_aux)
    imagen_res = np.array(array_aux)
    if mostrar:
        plt.imshow(imagen_res)
        plt.axis('off')
        plt.show()
    return imagen_res

# Recibe una imagen y un tamaño de ventana,
# Calcula la energia para cada ventana de la imagen
# Regresa un arreglo en forma de imagen concatenando los resultados de cada ventana.
def imagenWSmoothness(imagen,window_length,window_height,**kwargs):
    Y, X = imagen.shape
    array_aux = []
    length_overlap = (kwargs["length_overlap"] if ("length_overlap" in kwargs) else 0)
    height_overlap = (kwargs["height_overlap"] if ("height_overlap" in kwargs) else 0)
    mostrar = (kwargs["show"] if ("show" in kwargs) else False)
    for texel_renglon in range(0,Y,window_height-height_overlap):
        renglon_aux = []
        for texel_startP in range(0,X,window_length-length_overlap):
            ventana = imagen[texel_renglon:texel_renglon+window_height,texel_startP:texel_startP+window_length]
            glcm_N = glcmMaker_h(ventana,True)
            renglon_aux.append(smoothness(glcm_N))
        array_aux.append(renglon_aux)
    imagen_res = np.array(array_aux)
    if mostrar:
        plt.imshow(imagen_res)
        plt.axis('off')
        plt.show()
    return imagen_res

# Recibe una imagen y un tamaño de ventana,
# Calcula la contraste para cada ventana de la imagen
# Regresa un arreglo en forma de imagen concatenando los resultados de cada ventana.
def imagenWContrast(imagen,window_length,window_height,**kwargs):
    Y, X = imagen.shape
    array_aux = []
    length_overlap = (kwargs["length_overlap"] if ("length_overlap" in kwargs) else 0)
    height_overlap = (kwargs["height_overlap"] if ("height_overlap" in kwargs) else 0)
    mostrar = (kwargs["show"] if ("show" in kwargs) else False)
    for texel_renglon in range(0,Y,window_height-height_overlap):
        renglon_aux = []
        for texel_startP in range(0,X,window_length-length_overlap):
            ventana = imagen[texel_renglon:texel_renglon+window_height,texel_startP:texel_startP+window_length]
            glcm_N = glcmMaker_h(ventana,True)
            renglon_aux.append(contrast(glcm_N))
        array_aux.append(renglon_aux)
    imagen_res = np.array(array_aux)
    if mostrar:
        plt.imshow(imagen_res)
        plt.axis('off')
        plt.show()
    return imagen_res



In [5]:
# Recibe una imagen y un tamaño de ventana
# Calcula la entropia, homogeneidad, energia y contraste para cada ventana
# Devuelve un unico arreglo concatenando los resultados
def vectorizador(textura,window_l,window_h, **kwargs):
    l_over = (kwargs["length_overlap"] if ("length_overlap" in kwargs) else 0)
    h_over = (kwargs["height_overlap"] if ("height_overlap" in kwargs) else 0)
    mostrar = (kwargs["show"] if ("show" in kwargs) else False)
    entropia = imagenWEntropy(textura,window_l,window_h,length_overlap=l_over,height_overlap=h_over,show=mostrar)
    homogeneidad = imagenWHomogeneity(textura,window_l,window_h,length_overlap=l_over,height_overlap=h_over,show=mostrar)
    smoothness = imagenWSmoothness(textura,window_l,window_h,length_overlap=l_over,height_overlap=h_over,show=mostrar)
    contraste = imagenWContrast(textura,window_l,window_h,length_overlap=l_over,height_overlap=h_over,show=mostrar)
    return np.dstack((entropia,homogeneidad,smoothness,contraste))

### Transformación de datos

In [6]:
# Esta funcion recibe la "imagen" compuesta de caracteristicas y devuelve la lista de "pixeles de caracteristicas"
def pixelsImage(imagen):
    pixeles_clase = []
    for renglon in imagen:
        for pixel in renglon:
            pixeles_clase.append(pixel)
    return np.array(pixeles_clase)
#vectCaractTx1 = pixelsImage(imagenC1)

In [7]:
# Funcion auxiliar para mostrar el codigo de colores utilizado
def mostrarColores(colores, **kwargs):
    colormap = np.zeros((100,10,3), dtype=int)
    texturemap = np.zeros((100,10,3), dtype=int)
    fig, ax = plt.subplots()
    for c in range(len(colores)):
        colormap = np.concatenate((colormap,np.full((100,100,3), colores[c]),np.zeros((100,10,3), dtype=int)), axis=1)
        ax.text(50+(110*c),50,'T'+str(c+1), style ='italic', fontsize = 15)
        if ("texturas" in kwargs):
            texturemap = np.concatenate((texturemap,np.dstack((kwargs['texturas'][c][0:100,0:100],kwargs['texturas'][c][0:100,0:100],kwargs['texturas'][c][0:100,0:100])),np.zeros((100,10,3), dtype=int)), axis=1)
    if ("texturas" in kwargs):
        colormap = np.concatenate((colormap, texturemap))
    plt.axis('off')
    plt.imshow(colormap)
    plt.show()

***

### Clasificación con Bayes

#### Entrenamiento

In [8]:
# Esta función recibe una lista de las matrices de cada clase
def trainingBayess(matrices_clases):
    # Entrenamiento
    MUs = []
    SIGMAs_inv = []
    probas = []
    ldDers = []
    colores = []
    for clase in matrices_clases:
        MU = np.mean(clase, axis=0)
        MUs.append(MU)
        SIGMA = getSigma(clase)
        SIGMA_inv = np.linalg.inv(SIGMA)
        SIGMAs_inv.append(SIGMA_inv)
        proba = probaClase(len(clase), clase.shape[0], clase.shape[1], len(matrices_clases))
        probas.append(proba)
        ldDer = ladoDerecho(np.linalg.det(SIGMA), proba)
        ldDers.append(ldDer)
        colores.append((np.random.rand(3)*1000%255).astype(int).tolist())
    mostrarColores(colores)
    return MUs, SIGMAs_inv, ldDers, colores
#entrenamiento = trainingBayess([vectCaractTx1,vectCaractTx2,vectCaractTx3,vectCaractTx4,vectCaractTx5,vectCaractTx6])

***

### Clasificación con knn

In [9]:
# Preprocesamiento para ajustar con la funcion de knn de scikitlearn
def unificador(lista_vect, **kwargs):
    vect_array = []
    label_array = []
    for n in range(len(lista_vect)):
        for i in range(len(lista_vect[n])):
            vect_array.append(lista_vect[n][i])
            label_array.append( kwargs["labels"][n] if ("labels" in kwargs) else (n+1))
    return vect_array, label_array
#samp_feat, samp_lab = unificador([vectCaractTx1,vectCaractTx2,vectCaractTx3,vectCaractTx4,vectCaractTx5,vectCaractTx6])

#### Entrenamiento

In [10]:
model = KNeighborsClassifier()
#model.fit(samp_feat,samp_lab)

#### Transformación de datos

In [11]:
#vectIMG1 = pixelsImage(imgPC)

#### Pruebas

In [12]:
def mostrarKNN(clasificacion, forma, colores):
    resultado = np.zeros((forma[0],forma[1],3), dtype=int)
    aux = 0
    for i in range(len(resultado)):
        for j in range(len(resultado[i])):
            resultado[i][j]= colores[clasificacion[aux]-1]
            aux += 1
    plt.imshow(resultado)
    plt.axis('off')
    plt.show()
    return resultado

In [13]:
#clasificacion1 = model.predict(vectIMG1)


***

### Parte B

In [14]:
def rgb2gray(imagen):
    gris = ((imagen[:,:,0]+imagen[:,:,1]+imagen[:,:,2])/3).astype(int)
    return gris

### Conclusiones

Al haber implementado un clasificador de Bayes para la clasificacion de 2 o mas imagenes nos dimos cuenta de que con la aplicacion directa de la regla de Bayes da como resultado una problema computacionalmente costoso que no hace mas que incrementarse conforme la cantidad de imagenes de entrenamiento y la complejidad aumenta. Por tanto, si bien es un buen punto de partida, no es un clasificador optimo. Ademas, aprendimos que es necesario tener un buen volumen de imagenes de entrenamiento y que para ponerlo a prueba exhaustivamente las imagenes de entrenamiento y prueba deben diferir.


https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.slic
https://www.pyimagesearch.com/2014/07/28/a-slic-superpixel-tutorial-using-python/

### Referencias:

- (s.a) (s.f) matplotlib.pyplot.plot Documentacion de Mat-plotlib. Consultado de https://matplotlib.org/stable/api/as7gen/matplotlib.pyplot.plot.html

- (s.a) (s.f) matplotlib.contour.QuadContourSet Documentacion de Matplotlib. Consultado de https://matplotlib.org/stable/api/contourapi.html#matplotlib.contour.QuadContourSet

- (s.a) (s.f) matplotlib.image.AxesImage Documentacion de Matplotlib. Consultado de https://matplotlib.org/stable/api/imageapi.html#matplotlib.image.AxesImage

- (s.a) (s.f) matplotlib.pyplot.imshow Documentacion de Matplotlib. Consultado de https://matplotlib.org/stable/api/asgen/matplotlib.pyplot.imshow.html

- (s.a) (s.f) matplotlib.patches.Patch. Documentacion de Matplotlib. Consultado de https://matplotlib.org/stable/api/asgen/matplotlib.patches.Patch.html

- (s.a) (s.f) Shapely and geometric objects. Consultado de https://automating-gisprocesses.github.io/site/notebooks/L1/geometric-objects.html

- (s.a) (s.f) matplotlib.path. Documentacion de Matplotlib. Consultado de https://matplotlib.org/stable/api/pathapi.html

- (s.a) (s.f) matplotlib.pyplot.plot. Documentacion de Matplotlib. Consultado de https://matplotlib.org/stable/api/asgen/matplotlib.pyplot.plot.html

- (s.a) (s.f) Image Resolution and DPI. Consultado de https://largeprinting.com/resources/image-resolution-anddpi.html

- (s.a)(26 de dic, 2020) Apply a Gauss filter to an image with Python. Geeks for Geeks. Consultado de https://www.geeksforgeeks.org/apply-a-gauss-filter-to-animage-with-python/

- (s.a) (14 de julio, 2019) Python PIL GaussianBlur() method. Geeks for Geeks. Consultado
de https://www.geeksforgeeks.org/python-pil-gaussianblurmethod/

- Banterla, D. (s/f) Texturas. Fac. Informatica San Sebastian. Consultado de http://www.ehu.eus/ccwintco/uploads/d/d7/Texturas.pdf

- Dabbura, I. (17 de spetiembre, 2018) K-means Clustering: Algorithm, Applications, Evaluation Methods,and Drawbacks. Towards data science. Consultado de https://towardsdatascience.com/k-means-clusteringalgorithm-applications-evaluation-methods-and-drawbacksaa03e644b48a

- gene (13 de abril, 2017) Geopandas Polygon to matplotlib patches Polygon conversion Stack Exchange. Consultado de https://gis.stackexchange.com/questions/197945/geopandaspolygon-to-matplotlib-patches-polygon-conversion

- gene (4 de junio, 2014). Converting Matplotlib contour objects to Shapely objects. Stack Overflow. Consultado de https://gis.stackexchange.com/questions/99917/convertingmatplotlib-contour-objects-to-shapely-objects

- Ghandi, R. (5 de Mayo, 2018) Naive Bayes Classifier Towards Data Science. Consultado de https://towardsdatascience.com/naive-bayes-classifier81d512f50a7c

- Gillies, S.(27 de sep, 2020) The Shapely User Manual. Shapely. Consultado de https://shapely.readthedocs.io/en/stable/manual.html

- Hall-Beyer, M. (2017) GLCM Texture: A Tutorial v. 3.0. University of Calgary. Consultado de https://prism.ucalgary.ca/bitstream/handle/1880/51900/texture%20tutorial%20v%2030%20180206.pdf?sequence=11&isAllowed=y

- jodag. (6 de mayo, 2020) Matplotlib -unable to save image in same resolution as original image. Stack Overflow. Consultado de https://stackoverflow.com/questions/34768717/matplotlibunable-to-save-image-in-same-resolution-as-originalimage34769840

- Korstanje, J. (7 de abril, 2021) The k-Nearest Neighbors (kNN) Algorithm in Python. RealPython. Consultado en https://realpython.com/knn-python/

- Lin, W. et al. (2010) Image Segmentation Using the Kmeans Algorithm for Texture Features. World Academy of Science, Engineering and Technology International Journal of Computer and Information Engineering

- Navlani, A. (2 de agosto, 2018) KNN Classification using Scikit-learn. Datacamp. Consultado en https://www.datacamp.com/community/tutorials/k-nearestneighbor-classification-scikit-learn

- R, Kirsten et al. (5 se septiembre, 2019) Performance of two multiscale texture algorithms in classifying silver gelatine paper via k-nearest neighbors. Open Archive Toulouse Archive Ouverte. Consultado de https://hal.archives-ouvertes.fr/hal02279362/document

- Rosebrock, A. (8 de agosto, 2016) k-NN classifier for image classification. pyImageSearch. Consultado en https://www.pyimagesearch.com/2016/08/08/k-nn-classifierfor-image-classification/

- tom10 (23 de Marzo, 2015) Python - convert contours to image. Stack Overflow. Consultado de https://stackoverflow.com/questions/29213238/pythonconvert-contours-to-image2921417

- A. Rosebrock, "k-NN classifier for image classification - PyImageSearch", PyImageSearch, 2021. [Online]. Available: https://www.pyimagesearch.com/2016/08/08/k-nn-classifier-for-image-classification/. [Accessed: 26- Jul- 2021]. https://www.pyimagesearch.com/2016/08/08/k-nn-classifier-for-image-classification/

- scikit-learn, "sklearn.neighbors.KNeighborsClassifier — scikit-learn 0.24.2 documentation", Scikit-learn.org. [Online]. Available: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.fit. [Accessed: 26- Jul- 2021].